In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from IPython.display import HTML

def show_plot(fig):
    return HTML(fig.to_html(include_plotlyjs=False, full_html=False))

np.random.seed(42)


In [2]:
n = 120  # months
mkt = np.random.normal(0.006, 0.04, n)
smb = np.random.normal(0.002, 0.03, n)
hml = np.random.normal(0.002, 0.03, n)

true_alpha = 0.001
true_betas = np.array([1.1, 0.4, 0.2])
noise = np.random.normal(0.0, 0.02, n)

excess_ret = true_alpha + true_betas[0] * mkt + true_betas[1] * smb + true_betas[2] * hml + noise

X = np.column_stack([np.ones(n), mkt, smb, hml])
coef, *_ = np.linalg.lstsq(X, excess_ret, rcond=None)
alpha_hat = coef[0]
betas_hat = coef[1:]

df = pd.DataFrame({
    'MKT': mkt,
    'SMB': smb,
    'HML': hml,
    'Excess Return': excess_ret
})

alpha_hat, betas_hat


(0.002565914382535999, array([1.07486223, 0.36082155, 0.15714763]))

In [3]:
cum = (1 + df[['MKT', 'SMB', 'HML']]).cumprod() - 1

fig = go.Figure()
for col in cum.columns:
    fig.add_trace(go.Scatter(x=cum.index, y=cum[col], mode='lines', name=col))
fig.update_layout(
    title='Cumulative Factor Returns (Toy Data)',
    xaxis_title='Month',
    yaxis_title='Cumulative Return',
    template='plotly_white'
)
show_plot(fig)


In [4]:
labels = ['MKT', 'SMB', 'HML']
fig = go.Figure()
fig.add_trace(go.Bar(x=labels, y=betas_hat, name='Estimated'))
fig.add_trace(go.Bar(x=labels, y=true_betas, name='True', opacity=0.6))
fig.update_layout(
    title='Estimated vs True Betas',
    yaxis_title='Beta',
    barmode='group',
    template='plotly_white'
)
show_plot(fig)


In [5]:
pred = X @ coef
fig = go.Figure()
fig.add_trace(go.Scatter(x=pred, y=excess_ret, mode='markers', name='Actual vs Predicted'))
fig.update_layout(
    title='Predicted vs Actual Excess Returns',
    xaxis_title='Predicted',
    yaxis_title='Actual',
    template='plotly_white'
)
show_plot(fig)


In [6]:
resid = excess_ret - pred
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(n), y=resid, mode='lines', name='Residuals'))
fig.update_layout(
    title='Model Residuals',
    xaxis_title='Month',
    yaxis_title='Residual',
    template='plotly_white'
)
show_plot(fig)
